In [ ]:
import scipy.stats as stats
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from google.colab import files

In [ ]:
upload = files.upload()

Saving CBOE SKEW.xlsx to CBOE SKEW (1).xlsx


In [ ]:
vix_data = pd.read_excel('VIX.xlsx')
sp500_data = pd.read_excel('S&P 500.xlsx')
skew_data = pd.read_excel('CBOE SKEW (1).xlsx')
vix_data["Date"] = pd.to_datetime(vix_data["Date"])
sp500_data["Date"] = pd.to_datetime(sp500_data["Date"])
skew_data["Date"] = pd.to_datetime(skew_data["Date"])

# 1. Cleaning, Merging and Calculating Variance Risk Premium

In [ ]:
# Sort the S&P 500 data by date
sp500_data = sp500_data.sort_values("Date")

# Compute daily log returns
sp500_data["Log_Return"] = np.log(sp500_data["SPX"] / sp500_data["SPX"].shift(1))

# Compute rolling 30-trading-day realized volatility (annualized)
sp500_data["RV_30D"] = sp500_data["Log_Return"].rolling(window=30, min_periods=30).std() * np.sqrt(252)

# Shift realized volatility forward by 1 month to align with VIX dates
sp500_data["Date"] = sp500_data["Date"] + pd.DateOffset(months=1)

# Drop NaN values resulting from rolling calculation
sp500_data_cleaned = sp500_data.dropna(subset=["RV_30D"])

# Ensure both datasets are sorted before merging
sp500_data_cleaned = sp500_data_cleaned.dropna(subset=["Date"])
sp500_data_cleaned = sp500_data_cleaned.sort_values("Date")
vix_data_cleaned = vix_data.dropna(subset=["Date"])
vix_data_cleaned = vix_data_cleaned.sort_values("Date")

# Merge using nearest date matching (match as early as possible)
merged_data = pd.merge_asof(
    vix_data_cleaned,
    sp500_data_cleaned,
    on="Date",
    direction="backward",  # Match with the closest available older date
    suffixes=("_VIX", "_RV")
)
# Drop any remaining NaN values after merging
final_data = merged_data.dropna(subset=["RV_30D"])

In [ ]:
# Compute Variance Risk Premium (VRP)
final_data["IV"] = final_data["VIX"] / 100  # Convert VIX percentage to decimal
final_data["RV"] = final_data["RV_30D"]  # RV is already in decimal form

# Calculate VRP using the formula: VRP = IV^2 - RV^2
final_data["VRP"] = final_data["IV"]**2 - final_data["RV"]**2

In [ ]:
final_data

,Date,VIX,SPX,Log_Return,RV_30D,IV,RV,VRP
0,1990-01-02,17.24,353.73,0.008745,0.126449,0.1724,0.126449,0.013732
1,1990-01-03,18.19,353.73,0.008745,0.126449,0.1819,0.126449,0.017098
2,1990-01-04,19.22,353.73,0.008745,0.126449,0.1922,0.126449,0.020951
3,1990-01-05,20.11,352.56,-0.003313,0.124883,0.2011,0.124883,0.024846
4,1990-01-08,20.26,348.76,0.001176,0.121245,0.2026,0.121245,0.026346
...,...,...,...,...,...,...,...,...
8870,2025-02-24,18.98,5809.86,0.002143,0.094512,0.1898,0.094512,0.027092
8871,2025-02-25,19.43,5808.12,-0.000300,0.093781,0.1943,0.093781,0.028958
8872,2025-02-26,19.10,5808.12,-0.000300,0.093781,0.1910,0.093781,0.027686
8873,2025-02-27,21.13,5808.12,-0.000300,0.093781,0.2113,0.093781,0.035853


In [ ]:
# Forward-fill missing SKEW values to align with IV & VRP dataset
skew_data_ffill = skew_data.set_index("Date").asfreq("D").fillna(method="ffill").reset_index()
# Merge the forward-filled SKEW data with the IV & VRP dataset
merged_skew_data = pd.merge(final_data, skew_data_ffill, on="Date", how="left")
merged_skew_data

<ipython-input-16-1008c315a0d6>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  skew_data_ffill = skew_data.set_index("Date").asfreq("D").fillna(method="ffill").reset_index()


,Date,VIX,SPX,Log_Return,RV_30D,IV,RV,VRP,SKEW
0,1990-01-02,17.24,353.73,0.008745,0.126449,0.1724,0.126449,0.013732,126.09
1,1990-01-03,18.19,353.73,0.008745,0.126449,0.1819,0.126449,0.017098,123.34
2,1990-01-04,19.22,353.73,0.008745,0.126449,0.1922,0.126449,0.020951,122.62
3,1990-01-05,20.11,352.56,-0.003313,0.124883,0.2011,0.124883,0.024846,121.27
4,1990-01-08,20.26,348.76,0.001176,0.121245,0.2026,0.121245,0.026346,124.12
...,...,...,...,...,...,...,...,...,...
8870,2025-02-24,18.98,5809.86,0.002143,0.094512,0.1898,0.094512,0.027092,156.93
8871,2025-02-25,19.43,5808.12,-0.000300,0.093781,0.1943,0.093781,0.028958,157.60
8872,2025-02-26,19.10,5808.12,-0.000300,0.093781,0.1910,0.093781,0.027686,153.44
8873,2025-02-27,21.13,5808.12,-0.000300,0.093781,0.2113,0.093781,0.035853,149.32


In [ ]:
regime_data = merged_skew_data[["Date", "IV", "RV", "VRP", "SKEW"]]
regime_data

,Date,IV,RV,VRP,SKEW
0,1990-01-02,0.1724,0.126449,0.013732,126.09
1,1990-01-03,0.1819,0.126449,0.017098,123.34
2,1990-01-04,0.1922,0.126449,0.020951,122.62
3,1990-01-05,0.2011,0.124883,0.024846,121.27
4,1990-01-08,0.2026,0.121245,0.026346,124.12
...,...,...,...,...,...
8870,2025-02-24,0.1898,0.094512,0.027092,156.93
8871,2025-02-25,0.1943,0.093781,0.028958,157.60
8872,2025-02-26,0.1910,0.093781,0.027686,153.44
8873,2025-02-27,0.2113,0.093781,0.035853,149.32


In [ ]:
regime_data.to_csv("regime_data.csv", index=False)
files.download("regime_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>